In [6]:
# Import functions to load the model, create an adaptor for LoRA weights,
# and perform conditional text generation.

from fastlora.config import FastLoraConfig
from fastlora.model import FastLoraModelForCausalLM, FastLoraModel, get_peft_model_state_dict, set_peft_model_state_dict, load_pretrained_model
import peft.peft_model as peft_model
import peft.mapping as peft_mapping

## monkey patching
peft_model.PEFT_TYPE_TO_MODEL_MAPPING.update({"FASTLORA": FastLoraModel})
peft_mapping.PEFT_TYPE_TO_CONFIG_MAPPING.update({"FASTLORA": FastLoraConfig})
peft_model.get_peft_model_state_dict = get_peft_model_state_dict
peft_model.set_peft_model_state_dict = set_peft_model_state_dict

from fastlora.eval_utils import load_model_and_tokenizer
from fastlora.eval_utils import fastlora_generate_adaptor
from fastlora.eval_utils import fastlora_conditional_generate
from pathlib import Path
import json
import torch
from peft.config import PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer


In [7]:
# Define the path to the pretrained model checkpoint and the device to run on.
model_name_or_path = "generative-adaptor/Generative-Adapter-Mistral-7B-Instruct-v0.2"
device = 'cuda'
torch_dtype = torch.bfloat16
attn_implementation = 'sdpa'

In [ ]:
# Load the PEFT configuration from the given pretrained model directory.
peft_config = PeftConfig.from_pretrained(model_name_or_path)

# Get the base model path from the configuration.
base_model_path = peft_config.base_model_name_or_path

# Ensure that the base model path is available.
assert base_model_path is not None, "base_model_name_or_path should not be None"

# Load the base causal language model from the retrieved base model path.
# The model is loaded with a specific data type and attention implementation.
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    torch_dtype=torch_dtype,
    attn_implementation=attn_implementation,
)

# Load the FastLora model for causal language modeling.
# This model is built on top of the base model, uses the adapter settings from the PEFT config,
# is not set for training, and is moved to the GPU.
model = FastLoraModelForCausalLM.from_pretrained(
    base_model,
    model_name_or_path,
    adapter_name='default',
    is_trainable=False,
    config=peft_config,
).cuda()

# Load the tokenizer from the pretrained model directory.
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)


Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.11s/it]


peft_config FastLoraConfig(peft_type='FASTLORA', auto_mapping=None, base_model_name_or_path='mistralai/Mistral-7B-Instruct-v0.2', revision=None, task_type='FAST_LORA_CAUSAL_LM', inference_mode=True, fastlora_r=1024, fastlora_inter_size=1024, fastlora_window=1024, fastlora_max_rank=128, fastlora_attn_len=8192, fastlora_gist_len=0, fastlora_alpha=64.0, fastlora_dropout=0.0, fastlora_arch='aassbb', fastlora_norm='svd', fastlora_init='default', fastlora_merge='pre-norm-sum', fastlora_param=['o_proj'], fastlora_training_attention_mask='default', target_modules=['o_proj'], lora_r=0, lora_alpha=0, lora_dropout=0.0, lora_param=None, layer_replication=None, megatron_config=None, bias='none')
[WARNING] we apply a monkey patch to the function `get_peft_model_state_dict` to support FastLora model
missing_keys: []
peft_model_state_dict.keys(): dict_keys(['base_model.model.model.layers.0.base_layer.self_attn.o_proj.fastlora_A1.weight', 'base_model.model.model.layers.0.base_layer.self_attn.o_proj.fas

In [9]:
# Define a prompt prefix that supplies context about a speaker's background.
# This information may influence how the model generates the response.
prompt_prefix = (
    "I volunteer in my spare time. I have been volunteering for 7 years. I volunteer in a homeless shelter in my town. "
    "I'm not into cars. I wrestle for my day job. I like wrestling. I am not super into wrestling. I like crowds and meeting people. "
    "I work out a few times each week when I need to be alone. I like country music a little bit. I like Taylor Swift. "
    "I've lost fights recently. I work out a few times a week. I do not like working on cars. I am not patient. "
    "I have two dogs: Baron Zemo and Spike. I have two older mustangs. I like vintage cars. I'm working on two Mustangs: a 68 and a 66 Hertz clone. "
    "I've been working on cars since 1989. I have a Mustang convertible. I work on my car after work. I get frustrated working on my car sometimes. "
    "I don't like crowds. I like working out. I like classic country. I am a dog trainer. My work keeps me busy."
)

# Define the input prompt that asks a question related to music.
prompt_input = "Hey, remember that time we talked about music? What was the artist you mentioned you could get into?"

# Set parameters for generating the LoRA weights and the text output.
merge_strategy = 'concat'
window_size = 1024
max_new_tokens = 100
stop = ["\n"]

In [11]:
# Generate LoRA weights using the model, tokenizer, and prompt prefix.
# This function adapts the model weights based on the given prompt context.
lora_weights = fastlora_generate_adaptor(
    model, tokenizer, 
    prompt_prefix, 
    merge_strategy=merge_strategy, 
    max_window_size=window_size,
)

# Output the number of LoRA weights generated.
print("Number of LoRA weights generated:", len(lora_weights))

shape of context_input_ids: torch.Size([1, 1, 237])
shape of context_attention_mask: torch.Size([1, 1, 237])
Number of LoRA weights generated: 32


In [13]:
# Generate text using the model with the generated LoRA weights.
# The function takes the input prompt and uses chat-style generation.
output_text = fastlora_conditional_generate(
    model, tokenizer, 
    input_text=prompt_input, 
    use_chat=True,
    mode="weights", 
    lora_weights=lora_weights, 
    max_new_tokens=max_new_tokens,
    stop=stop,
)

# Print the generated text.
print(output_text)

I like Taylor Swift.
